In [1]:
#从数据库获取数据 更新toh_get_ toh_use相关原始数据
import sys
import os
import codecs
from sys import argv
import pymysql
import pandas as pd

config = dict(host='125.124.0.156', user='chencheng', password='Cc$toh$db1', port= 3306,database = "toh", cursorclass=pymysql.cursors.DictCursor, charset='UTF8')

conn = pymysql.Connect(**config)
conn.autocommit(1)
cursor = conn.cursor()

dic = {};

def run_out():
    sql_ = '''select `desc`,system,channel from
(select channel,`desc` from
	(select * from (select distinct other, type from `toh_use_316`) result
		left join toh_via 
		on result.other = toh_via.id_) result2
left join toh_res on result2.type = toh_res.id) result3
left join toh_via_func on result3.channel = toh_via_func.`function`
    '''
    cursor.execute(sql_)
    all_ret = cursor.fetchall()
    out_data = pd.DataFrame(all_ret);
    print(out_data)
    out_data.to_csv("toh_use.csv", index=False);
    del out_data['channel'];
    new = out_data.drop_duplicates(subset=['desc','system'])
    new.to_csv("toh_use2.csv", index=False);


def run_get():
    sql_ = '''select `desc`,system,channel from
(select channel,`desc` from
	(select * from (select distinct other, type from `toh_get_316_copy`) result
		left join toh_via 
		on result.other = toh_via.id_) result2
left join toh_res on result2.type = toh_res.id) result3
left join toh_via_func on result3.channel = toh_via_func.`function`
    '''
    cursor.execute(sql_)
    all_ret = cursor.fetchall()
    out_data = pd.DataFrame(all_ret);
    print(out_data)
    out_data.to_csv("toh_get.csv", index=False);
    del out_data['channel'];
    new = out_data.drop_duplicates(subset=['desc','system'])
    new.to_csv("toh_get2.csv", index=False);

if __name__ == '__main__':
    '''
    main
    '''
    run_get();
    run_out();

          desc system             channel
0           金币    周活动           顺序兑换活动活动赠
1         时光之沙    周活动           顺序兑换活动活动赠
2           卡牌    周活动           顺序兑换活动活动赠
3     固定英雄卡牌碎片    周活动           顺序兑换活动活动赠
4           装备    周活动           顺序兑换活动活动赠
..         ...    ...                 ...
535    骰子 dice    周活动  登陆有奖(活动期间登录拿东西)活动赠
536        虚空石    周活动  登陆有奖(活动期间登录拿东西)活动赠
537  lemon_tea    周活动  登陆有奖(活动期间登录拿东西)活动赠
538  stopwatch    周活动  登陆有奖(活动期间登录拿东西)活动赠
539   egg_tart    周活动  登陆有奖(活动期间登录拿东西)活动赠

[540 rows x 3 columns]
                      desc system      channel
0                       钻石    周活动    顺序兑换活动活动赠
1                      灵魂石    周活动    顺序兑换活动活动赠
2                lemon_tea    周活动    顺序兑换活动活动赠
3                stopwatch    周活动    顺序兑换活动活动赠
4    horse searach stamina     龙岛         天马探索
..                     ...    ...          ...
142                    许愿币     转盘         使用转盘
143                     钻石     工会         创建公会
144                 进背包的宝箱    周活动          开宝箱
14

In [11]:
#从toh_get_ toh_use相关原始数据 整理出作图需要的out.csv数据
import networkx as nx
import pandas as pd

l_source = set({});
l_funct = set({})

df = pd.read_csv('./toh_use2.csv')
df.rename(columns={'desc': 'source', 'system' : 'target'}, inplace=True);
Graphtype = nx.DiGraph()
G = nx.from_pandas_edgelist(df, create_using=Graphtype)
list(G)

list(G.edges())
for node in list(G):
    nbrs = nx.neighbors(G, node)
#     print(node, list(set(nbrs)));
    if(len(set(nbrs)) == 0):
        l_funct.add(node);
    else:
       #是资源类型
        l_source.add(node);

df2 = pd.read_csv('./toh_get2.csv')
df2.rename(columns={'desc': 'target', 'system' : 'source'}, inplace=True);
Graphtype = nx.DiGraph()
G2 = nx.from_pandas_edgelist(df2, create_using=Graphtype)
list(G2.edges())
for node in list(G2):
    nbrs = nx.neighbors(G2, node)
#     print(node, list(set(nbrs)));
    if(len(set(nbrs)) == 0):
       #是资源类型
       l_source.add(node);
    else:
        l_funct.add(node);
       #是功能类型

# 只取某系统/某资源对应系统
U_all=nx.DiGraph()
U_all.add_edges_from(G.edges())
U_all.add_edges_from(G2.edges());
U_all.add_nodes_from(G.nodes())
U_all.add_nodes_from(G2.nodes())
U_all.remove_node('周活动');
U_all.remove_node('月活动');
U_all.remove_node('月礼包');
U_all.remove_node('礼包');
l_funct.remove('周活动');
l_funct.remove('月活动');
l_funct.remove('月礼包');
l_funct.remove('礼包');


# print("all system:", list(l_funct));

def out_csv(_U_all, out_path = "to_io_draw.csv"):
    df_out = pd.DataFrame([], columns=['id','name','shape','fill','refs']);
    i = 0;
    for node in list(_U_all):
        nbrs = nx.neighbors(_U_all, node)
        shape_ = "ellipse" if(node in l_source) else "rectangle"
        fill_ = "#b85450" if(node in l_source) else "#82b366"
        row = [node, node, shape_, fill_ ,','.join(list(set(nbrs)))]
        df_out.loc[i]=row
        i+=1;
    df_out.to_csv(out_path, index=False);

for s in l_source:
    subnodes = list(nx.all_neighbors(U_all, s)) + [s];
    print(subnodes)
    sub = U_all.subgraph(subnodes)
    print("edgess " + s, list(sub.nodes), list(sub.edges));    
    out_csv(sub, "./draw_io_out/source/to_io_draw_" + s + ".csv");

for s in l_funct:
    subnodes = list(nx.all_neighbors(U_all, s)) + [s];
    print(subnodes)
    sub = U_all.subgraph(subnodes)
    print("edgess " + s, list(sub.nodes), list(sub.edges));    
    out_csv(sub, "./draw_io_out/system/to_io_draw_sys_" + s + ".csv");
#     print(row)
#     new_row = pd.DataFrame(, columns=['id','name','refs']);
#     print(new_row)
#     df_out.append(new_row, ignore_index=True)



['先知之树', '高级置换材料 荷鲁斯之眼']
edgess 高级置换材料 荷鲁斯之眼 ['高级置换材料 荷鲁斯之眼', '先知之树'] [('高级置换材料 荷鲁斯之眼', '先知之树')]
['地牢', '地牢', 'LOSTCITY_COIN']
edgess LOSTCITY_COIN ['地牢', 'LOSTCITY_COIN'] [('地牢', 'LOSTCITY_COIN'), ('LOSTCITY_COIN', '地牢')]
['封印之地', '封印之地', '封印之地战斗次数']
edgess 封印之地战斗次数 ['封印之地战斗次数', '封印之地'] [('封印之地战斗次数', '封印之地'), ('封印之地', '封印之地战斗次数')]
['好友', '好友', '好友boss体力，新的3']
edgess 好友boss体力，新的3 ['好友', '好友boss体力，新的3'] [('好友', '好友boss体力，新的3'), ('好友boss体力，新的3', '好友')]
['竞技场', '工会', '新手养成', '地牢', '龙岛', '活跃活动', '好友', '远征', '爬塔', '成就系统', '酒馆', '运营处理', '工会', '背包系统', '竞技场', '基础商店', '封印之地', '转盘', '其他养成', '龙岛', '地牢', '远征', '爬塔', '运营处理', '抽卡', '酒馆', '装备系统', '个人信息', '钻石']
edgess 钻石 ['装备系统', '地牢', '基础商店', '活跃活动', '封印之地', '竞技场', '转盘', '新手养成', '好友', '个人信息', '酒馆', '抽卡', '龙岛', '成就系统', '工会', '背包系统', '运营处理', '远征', '爬塔', '其他养成', '钻石'] [('地牢', '钻石'), ('活跃活动', '钻石'), ('竞技场', '钻石'), ('新手养成', '钻石'), ('好友', '钻石'), ('酒馆', '钻石'), ('龙岛', '钻石'), ('成就系统', '钻石'), ('工会', '钻石'), ('运营处理', '钻石'), ('远征', '钻石'), ('爬塔', '钻石'), ('钻石', '工会

['竞技场', '工会', '地牢', '龙岛', '转盘', '好友', '酒馆', '征战大厅', '装备系统', '神器碎片']
edgess 神器碎片 ['工会', '酒馆', '装备系统', '竞技场', '地牢', '龙岛', '转盘', '征战大厅', '好友', '神器碎片'] [('工会', '神器碎片'), ('酒馆', '神器碎片'), ('竞技场', '神器碎片'), ('地牢', '神器碎片'), ('龙岛', '神器碎片'), ('转盘', '神器碎片'), ('征战大厅', '神器碎片'), ('好友', '神器碎片'), ('神器碎片', '装备系统')]
['竞技场', '工会', '地牢', '基础商店', '龙岛', '先知之树', '转盘', '分解功能', '爬塔', '英雄系统', '成就系统', '副本系统', '征战大厅', '英雄系统', '卡牌进阶石头']
edgess 卡牌进阶石头 ['工会', '竞技场', '地牢', '英雄系统', '龙岛', '基础商店', '先知之树', '转盘', '爬塔', '副本系统', '征战大厅', '分解功能', '成就系统', '卡牌进阶石头'] [('工会', '卡牌进阶石头'), ('竞技场', '卡牌进阶石头'), ('地牢', '卡牌进阶石头'), ('英雄系统', '卡牌进阶石头'), ('龙岛', '卡牌进阶石头'), ('基础商店', '卡牌进阶石头'), ('先知之树', '卡牌进阶石头'), ('转盘', '卡牌进阶石头'), ('爬塔', '卡牌进阶石头'), ('副本系统', '卡牌进阶石头'), ('征战大厅', '卡牌进阶石头'), ('分解功能', '卡牌进阶石头'), ('成就系统', '卡牌进阶石头'), ('卡牌进阶石头', '英雄系统')]
['龙岛', '征战大厅', '酒馆', '酒馆卷轴']
edgess 酒馆卷轴 ['酒馆', '征战大厅', '龙岛', '酒馆卷轴'] [('征战大厅', '酒馆卷轴'), ('龙岛', '酒馆卷轴'), ('酒馆卷轴', '酒馆')]
['远征', '成就系统', '远征', '远征币']
edgess 远征币 ['远征币', '远征', '成就系统'] [('远征币', '远征'), ('远征

['钻石', '金币', '公会币', '固定英雄卡牌碎片', '随机英雄碎片', '金币', '公会币', '固定英雄卡牌碎片', '随机英雄碎片', '时光之沙', '卡牌升级石头', '卡牌进阶石头', '钻石', 'stopwatch', '装备', '神器碎片', '工会']
edgess 工会 ['工会', '钻石', '公会币', '时光之沙', '卡牌升级石头', '随机英雄碎片', '金币', '装备', '神器碎片', 'stopwatch', '固定英雄卡牌碎片', '卡牌进阶石头'] [('工会', '金币'), ('工会', '公会币'), ('工会', '固定英雄卡牌碎片'), ('工会', '随机英雄碎片'), ('工会', '时光之沙'), ('工会', '卡牌升级石头'), ('工会', '卡牌进阶石头'), ('工会', '钻石'), ('工会', 'stopwatch'), ('工会', '装备'), ('工会', '神器碎片'), ('钻石', '工会'), ('公会币', '工会'), ('随机英雄碎片', '工会'), ('金币', '工会'), ('固定英雄卡牌碎片', '工会')]
['钻石', '固定英雄卡牌碎片', '随机英雄碎片', '指定皮肤碎片(指定的skin id)', '随机皮肤碎片', '卡牌', '皮肤', '背包系统']
edgess 背包系统 ['背包系统', '随机皮肤碎片', '随机英雄碎片', '固定英雄卡牌碎片', '皮肤', '指定皮肤碎片(指定的skin id)', '卡牌', '钻石'] [('背包系统', '卡牌'), ('背包系统', '皮肤'), ('随机皮肤碎片', '背包系统'), ('随机英雄碎片', '背包系统'), ('固定英雄卡牌碎片', '背包系统'), ('指定皮肤碎片(指定的skin id)', '背包系统'), ('钻石', '背包系统')]
['钻石', '金币', '固定英雄卡牌碎片', '先知宝珠，先知门', '抽卡卷轴', '许愿币', '高级许愿币', '钻石', '运营处理']
edgess 运营处理 ['高级许愿币', '运营处理', '许愿币', '固定英雄卡牌碎片', '金币', '抽卡卷轴', '先知宝珠，先知门', '钻石'] [('高

In [ ]:
test_dup = pd.read_csv("toh_get2.csv");
print(test_dup);
g = test_dup.groupby(['desc','system']).groups
print(len(g))
# test_dup.drop_duplicates();
# print(test_dup)

In [ ]:
df_get = pd.read_csv('./toh_get.csv');
# print(df_get.describe)
set_get = set(df_get['source'])
# print(len(set_get));

df_use = pd.read_csv('./toh_use.csv');
# print(df_use.describe)
set_use = set(df_use['source'])
# print(len(set_use));
func_ = pd.read_csv('./func_system.csv');
sys = func_["system"];
ss_sys = set(sys);
print(len(ss_sys));
list(ss_sys)
ss_all = set(list(set_get) + list(set_use));
print(ss_all)
print(len(ss_all))